In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
import pickle as pkl
import torchvision
from attack import attack, test_model,parse_param


In [2]:
from datasets import load_cifar10, load_cifar100
from models.resnet import load_cifar10_resnet50, load_cifar100_resnet50
model = load_cifar100_resnet50()

In [3]:
all_param_names = list()
for name, param in model.named_parameters():
    all_param_names.append(name)

In [4]:
train_loaders, test_dataloaders, test_dataloader_all = load_cifar100()
all_totals = list()
for i in range(100):
    all_totals.append(attack(train_loaders[i], all_param_names, load_cifar100_resnet50, alpha=0.00002))


Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 2/2 [00:10<00:00,  5.50s/it]


0.00318152779340744


100%|██████████| 2/2 [00:01<00:00,  1.57it/s]


0.18206711578369142


...

4.35391259765625


100%|██████████| 2/2 [00:00<00:00,  2.78it/s]


10.05649560546875


100%|██████████| 2/2 [00:00<00:00,  2.87it/s]


14.53730029296875


In [5]:
pkl.dump(all_totals, open("weights/cifar100_resnet50_totals.pkl", "wb"))

In [14]:
thre = 0.2
net = load_cifar100_resnet50()
param_remove = dict()
for param in all_param_names:
    param_remove[param] = None
for i in range(len(all_totals)):
    totals = all_totals[i]
    totals = [totals[param] for param in all_param_names]
    param_weights = [eval("net." + parse_param(param) + ".cpu().detach().numpy()")
                     for param in all_param_names]
    combine = [np.abs(total * weight) for total, weight in zip(totals, param_weights)]
    combine = np.array(combine)
    combine_flatten = np.concatenate([combine_.flatten() for combine_ in combine],axis=0)
    threshold = np.sort(combine_flatten)[::-1][int(len(combine_flatten) * thre)]
    for idx,param in enumerate(all_param_names):
        if param_remove[param] is None:
            param_remove[param] = combine[idx] > threshold
        else:
            t = combine[idx] > threshold
            param_remove[param] = param_remove[param] | t

C:\Users\Zhang\AppData\Local\Temp\ipykernel_30180\3212750293.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  combine = np.array(combine)


In [15]:
temp = 0
all_num = 0
for param in param_remove:
    temp += param_remove[param].sum()
    all_num += param_remove[param].size
    print(param, param_remove[param].mean())

conv1.weight 0.9895833333333334
bn1.weight 1.0
bn1.bias 1.0
layer1.0.conv1.weight 0.7763671875
layer1.0.bn1.weight 1.0
layer1.0.bn1.bias 0.984375
layer1.0.conv2.weight 0.5720757378472222
layer1.0.bn2.weight 1.0
layer1.0.bn2.bias 1.0
layer1.0.conv3.weight 0.69427490234375
layer1.0.bn3.weight 0.953125
layer1.0.bn3.bias 0.9765625
layer1.0.shortcut.0.weight 0.71368408203125
layer1.0.shortcut.1.weight 0.953125
layer1.0.shortcut.1.bias 0.9765625
layer1.1.conv1.weight 0.56158447265625
layer1.1.bn1.weight 1.0
layer1.1.bn1.bias 0.984375
layer1.1.conv2.weight 0.5562065972222222
layer1.1.bn2.weight 1.0
layer1.1.bn2.bias 0.984375
layer1.1.conv3.weight 0.61761474609375
layer1.1.bn3.weight 0.95703125
layer1.1.bn3.bias 0.9609375
layer1.2.conv1.weight 0.52606201171875
layer1.2.bn1.weight 1.0
layer1.2.bn1.bias 0.953125
layer1.2.conv2.weight 0.5472547743055556
layer1.2.bn2.weight 1.0
layer1.2.bn2.bias 0.984375
layer1.2.conv3.weight 0.43963623046875
layer1.2.bn3.weight 0.8828125
layer1.2.bn3.bias 0.97656

In [16]:
temp / all_num

0.36506167494022

In [17]:
with torch.no_grad():
    net = load_cifar100_resnet50()
    correct, all = test_model(net, test_dataloader_all)
    print("原始准确率", correct / all)


原始准确率 0.7931


In [18]:
with torch.no_grad():
    net = load_cifar100_resnet50()
    for param in all_param_names:
        param_ = parse_param(param)
        try:
            exec("net." + param_ + "[~param_remove[param]] = 0")
        except:
            exec("net." + param_ + "[~param_remove[param],:] = 0")
    correct, all = test_model(net, test_dataloader_all)
    print("现在准确率", correct / all)

现在准确率 0.7328


In [19]:
with torch.no_grad():
    net = load_cifar100_resnet50()
    for param in all_param_names:
        param_ = parse_param(param)
        keep_rate = param_remove[param].sum() / param_remove[param].size
        weight_flatten = eval("net." + param_ + ".cpu().detach().numpy()").flatten()
        threshold = np.sort(weight_flatten)[int(len(weight_flatten) * (1 - keep_rate))]
        try:
            exec("net." + param_ + "[eval('net.' + param_ + '.cpu().detach().numpy()') < threshold] = 0")
        except:
            exec("net." + param_ + "[eval('net.' + param_ + '.cpu().detach().numpy()') < threshold,:] = 0")
    correct, all = test_model(net, test_dataloader_all)
    print("对比实验准确率", correct / all)

对比实验准确率 0.01
